# Algoritmo de Seleção de Endereços - Projeto Uber


## Objetivo
O objetivo deste algoritmo é gerar um conjunto de pares de endereços na cidade de São Paulo para o treinamento de um modelo de precificação de corridas de Uber.

## Metodologia
O algoritmo terá natureza estocástica, dando preferência para endereços de chegada em distritos com maior densidade populacional e renda média por habitante (fatores relevantes para o uso do aplicativo Uber).

Para os endereços de saída, o algoritmo usará uma função de decaimento exponencial $P(d)$ (descrita abaixo) para gerar uma distância máxima para o endereço do destino, onde escolherá o endereço de destino uniformemente dentro desse intervalo.


Usaremos o conjunto de dados geográficos (disponível como um arquivo `.shape` com um grafo representando as ruas de São Paulo) distribuído pela plataforma [GeoSampa](https://geosampa.prefeitura.sp.gov.br) da Prefeitura de São Paulo para acessar os endereços disponíveis dentro da distância $d$ da origem.

## WIP

TODOs:
- Unificar .shape de Logradouros e Distritos
- Checar a necessidade de titulos e prep no logradouro (ver comentario)
<!--
print(gdf['lg_titulo'].unique())
Reescrevendo os títulos em extenso 
 gdf['lg_titulo'] = gdf['lg_titulo'].replace({
     'ENG': 'Engenheiro',
     'PROF': 'Professor'
     ''
 })
-->


## Results
Describe and comment the most important results.

## Suggested next steps
State suggested next steps, based on results obtained in this notebook.

### Implementação em Código Python

A implementação segue as etapas principais descritas abaixo:

1. __Carregamento e Preparação dos Dados:__

   - Utilizaremos a biblioteca `geopandas` para manipular arquivos `.shapefile`.
   - 
2. __Cálculo das Probabilidades de Escolha:__

   - Importe os dados do IBGE utilizando `pandas`.
   - Calcularemos a probabilidade de um distrito $\bar{d}$ com base na população e renda dos distritos.
   $$
   \text{Probabilidade}(\bar{d}) = \frac{\text{População}(\bar{d}) \cdot \text{RendaMédia}(\bar{d})}{\sum^{distritos}_{d} (\text{População}(d) \cdot \text{RendaMédia}(d))}
   $$
3. __Seleção do Endereço de Origem:__

   - Escolha aleatória de uma rua com `numpy.random.choice`.
   - Geração de um número dentro do intervalo válido.
4. __Seleção do Endereço de Destino:__

   - Use $P(r)$ para calcular o raio máximo:

     $$
     P(r) = \frac{1}{\sqrt{2\pi}\sigma} e^{-\frac{r^2}{2\sigma^2}}
     $$
   - Escolha uniformemente dentro do raio usando coordenadas do grafo.
5. **Visualização dos Resultados:**

   - Use `matplotlib` ou `folium` para criar mapas interativos.





### Visualização e Validação

Após a implementação, as corridas simuladas podem ser visualizadas em mapas interativos:

- **Mapas com `folium`:** Representam pares de origem e destino.
- **Gráficos:** Analisam estatisticamente a distribuição das distâncias e destinos.

Além disso, as corridas geradas podem ser exportadas para arquivos `.csv` para uso em modelos de treinamento.

# Setup

## Importar bibliotecas
Importamos todas as bibliotecas Python necessárias

In [21]:
# Manipulação de dados
import pandas as pd
import numpy as np
import geopandas as gpd

# Opções para exibição de dados
pd.options.display.max_columns = 50
pd.options.display.max_rows = 30

# Visualização de dados
import matplotlib as plt
    


# Importação de Dados
Importaremos todos os dados necessários para a análise.

In [66]:
# Carregar os dados do IBGE
df_pop = pd.read_csv('data/population_parameters_rates_by_district.csv')
df_renda = pd.read_csv('data/renda_media_distrito.csv')

# Carregar os dados do geo-dataframe da cidade de São Paulo
gdf = gpd.read_file('/home/sasinhe/uber/data/mapas/SaoPaulo_merged/sao_paulo.shp')



# Processamento de Dados

## Tratamento dso arquivos do IBGE

O resultado dessa seção deve ser um dataframe `df_prob` com os nomes dos distritos e suas respectivas probabilidades

In [113]:
# Remover espaços em branco e converter para minúsculas
df_pop['Nome_distrito'] = df_pop['Nome_distrito'].str.strip().str.lower()
df_renda['Nome_distrito'] = df_renda['Nome_distrito'].str.strip().str.lower()

# Definir intervalos de renda e seus pontos médios
income_intervals = {
    'Menos de 2 SM': 1,
    'De 2 a Menos de 5 SM': 3.5,
    'De 5 a Menos de 10 SM': 7.5,
    'De 10 a Menos de 15 SM': 12.5,
    'De 15 a Menos de 25 SM': 20,
    'De 25 SM e Mais ': 30
}

# Calcular a renda média para cada distrito
for col, midpoint in income_intervals.items():
    df_renda[col] = df_renda[col]/100 * midpoint


df_renda['average_income'] = df_renda[list(income_intervals.keys())].sum(axis=1)

# Normalizar a renda média
min_income = df_renda['average_income'].min()
max_income = df_renda['average_income'].max()
df_renda['normalized_income'] = (df_renda['average_income'] - min_income) / (max_income - min_income)

# Normalizar a população
min_pop = df_pop['Pop_2020'].min()
max_pop = df_pop['Pop_2020'].max()
df_pop['normalized_pop'] = (df_pop['Pop_2020'] - min_pop) / (max_pop - min_pop)

# Mesclar os dois dataframes na coluna 'Nome_distrito'
df_prob = pd.merge(df_pop, df_renda, on='Nome_distrito')

# Calcular a pontuação composta (média simples das normalizações)
df_prob['composite_score'] = (df_prob['normalized_income'] + df_prob['normalized_pop']) / 2

# Normalizar a pontuação composta para obter probabilidades
total_score = df_prob['composite_score'].sum()
df_prob['Probabilidade'] = df_prob['composite_score'] / total_score

# Criar um novo dataframe com os resultados para checagem
# result = df.copy()

# Remover colunas desnecessárias
df_prob = df_prob[["Nome_distrito", "Probabilidade"]]

# Exibir o resultado final com as probabilidades
df_prob

,Nome_distrito,Probabilidade
0,agua rasa,0.009768
1,alto de pinheiros,0.014637
2,anhanguera,0.005868
3,aricanduva,0.007474
4,artur alvim,0.007562
...,...,...
90,vila medeiros,0.008441
91,vila prudente,0.009580
92,vila sonia,0.013279
93,sao domingos,0.008548


## Tratamento de arquivos .shp (mapas)

O resultado final deve ser um geo-dataframe `df_mapa` com as informações de cada logradouro

In [115]:
# Carregar os dados do geo-dataframe da cidade de São Paulo
gdf = gpd.read_file('/home/sasinhe/uber/data/mapas/SaoPaulo_merged/sao_paulo.shp')

gdf.rename(columns={'ds_nomeds_': 'nome_distrito', 'lg_nome': 'nome_logradouro'}, inplace=True) # Renomear colunas
gdf = gdf.loc[gdf['lg_tipo'].isin(['R', 'AV'])] # Filtrar apenas ruas e avenidas
gdf['lg_tipo'] = gdf['lg_tipo'].map({'R' : 'Rua', 'AV': 'Avenida'}) # Mapear códigos para nomes
gdf['num_ini'] = gdf[['lg_ini_par', 'lg_ini_imp']].min(axis=1) # Definir o número inicial
gdf['num_fim'] = gdf[['lg_fim_par', 'lg_fim_imp']].max(axis=1) # Definir o número final
gdf = gdf.dropna(subset=['num_fim']) # Remover valores nulos
gdf = gdf.dropna(subset=['num_ini']) # Remover valores nulos	
gdf[['num_fim', 'num_ini']] = gdf[['num_fim', 'num_ini']].astype(int) # Converter para inteiros
gdf.drop(columns=['lg_ini_par', 'lg_ini_imp', 'lg_fim_par', 'lg_fim_imp', 'lg_codlog', 'lg_or_geom', 'lg_seg_id', 'lg_id', 'lg_ordem'], inplace=True) # Remover colunas desnecessárias

gdf['logradouro'] = gdf['lg_tipo'] + ' ' + gdf['lg_titulo'].fillna('') + ' ' + gdf['lg_prep'].fillna('') + ' ' + gdf['nome_logradouro']

gdf = gdf.applymap(lambda s: s.lower() if type(s) == str else s) # Deixa todas as entradas que forem string minusculas

/tmp/ipykernel_10360/3962777102.py:16: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  gdf = gdf.applymap(lambda s: s.lower() if type(s) == str else s) # Deixa todas as entradas que forem string minusculas


# Criação do Algoritmo


## Endereço de Origem

In [107]:
def selecionar_distrito_de_origem(prob_distrito_de_origem=df_prob, geodataframe=gdf):
    distritos = prob_distrito_de_origem['Nome_distrito']
    probabilidades = prob_distrito_de_origem['Probabilidade']
    distrito_amostrado = np.random.choice(distritos, p=probabilidades)
    # print(f"Distrito amostrado: {distrito_amostrado}")
    df_ruas = geodataframe[geodataframe['nome_distrito'] == distrito_amostrado]
    return df_ruas

In [123]:
def selecionar_endereço_de_origem():
    df_ruas = selecionar_distrito_de_origem()
    rua_amostrada = df_ruas.sample()
    numero_amostrado = np.random.randint(rua_amostrada['num_ini'].values[0], rua_amostrada['num_fim'].values[0])
    print(f"Rua amostrada: {rua_amostrada['logradouro'].values[0]}")
    return str(rua_amostrada['logradouro'].values[0]) + ', ' + str(numero_amostrado) + ', São Paulo SP'

In [124]:
selecionar_endereço_de_origem()

Distrito amostrado: cidade lider
Rua amostrada: rua   rafael da silva e sousa


'rua   rafael da silva e sousa, 462, São Paulo SP'

## Endereço de Destino

#### Cálculo da Variável Aleatória $r$
O raio $r$ da distância máxima do endereço de chegada é calculado a partir da distribuição gaussiana:


$$
P(r) = \frac{1}{\sqrt{2\pi}\sigma} e^{-\frac{r^2}{2\sigma^2}}
$$

In [ ]:

def selecionar_endereço_de_destino():

# References
We report here relevant references:
1. author1, article1, journal1, year1, url1
2. author2, article2, journal2, year2, url2

In [110]:
df_ruas = selecionar_distrito_de_origem()
df_ruas

Distrito amostrado: jabaquara


,lg_tipo,lg_titulo,lg_prep,nome_logradouro,nome_distrito,geometry,num_ini,num_fim,logradouro
51636,avenida,None,None,durval pinto ferreira,jabaquara,"LINESTRING (330936.52 7382264.369, 330948.205 ...",858,1001,avenida durval pinto ferreira
51637,avenida,eng,None,armando de arruda pereira,jabaquara,"LINESTRING (333096.272 7381662.245, 333099.326...",4888,4922,avenida eng armando de arruda pereira
51638,rua,jorn,None,claudio abramo,jabaquara,"LINESTRING (333185.761 7385747.942, 333186.815...",0,67,rua jorn claudio abramo
51639,rua,None,None,delmira agustini,jabaquara,"LINESTRING (333316.793 7381993.515, 333317.257...",284,363,rua delmira agustini
51640,rua,cons,None,elias de carvalho,jabaquara,"LINESTRING (331042.909 7383471.996, 331069.396...",698,765,rua cons elias de carvalho
...,...,...,...,...,...,...,...,...,...
55296,rua,dr,None,vitor eugenio do sacramento,jabaquara,"LINESTRING (331898.502 7383978.964, 331904.694...",0,87,rua dr vitor eugenio do sacramento
55297,rua,dr,None,vitor eugenio do sacramento,jabaquara,"LINESTRING (331704.981 7383941.363, 331716.031...",250,271,rua dr vitor eugenio do sacramento
55298,rua,dr,None,vitor eugenio do sacramento,jabaquara,"LINESTRING (331642.298 7383896.22, 331648.731 ...",270,333,rua dr vitor eugenio do sacramento
55299,rua,None,None,edgar amorim amaral,jabaquara,"LINESTRING (331735.787 7383944.718, 331749.98 ...",0,181,rua edgar amorim amaral
